# Purpose
calculate delay time

In [1]:
import glob
import h5py
import numpy as np
import os

%matplotlib inline

## functions

In [2]:
def print_h5_tree(val, pre=''):
    if isinstance(val, h5py._hl.files.File):
        print(val.filename)

    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if isinstance(val, h5py._hl.group.Group):
                print(pre + '└── ' + key)
                print_h5_tree(val, pre+'    ')
            else:
                print(pre + '└── ' + key)
        else:
            if isinstance(val, h5py._hl.group.Group):
                print(pre + '├── ' + key)
                print_h5_tree(val, pre+'│   ')
            else:
                print(pre + '├── ' + key)


def print_h5_tree_from_path(fpath: str):
    with h5py.File(fpath, "r") as tree:
        print_h5_tree(tree)

In [3]:
def add_params(group, key, values):
    if values.get('value') is not None:
        _ = group.create_dataset(key, data=values['value'])
        return group
    group_ = group.create_group(f'{key}')
    for key_, values_ in values.items():
        group_ = add_params(group_, key_, values_)
    return group


def output(dstpath: str, output_dict: dict):
    with h5py.File(dstpath, 'w') as tree:
        for key in output_dict.keys():
            tree = add_params(tree, key, output_dict[key])

In [4]:
def calculate_timing_monitor_info(tree, speed_of_disordering, radius_init):
    """
    Parameters
    ----------
    speed_of_disordering : float
        speed of disordering (m/s = 1e-6 nm/fs).
    """
    radius_ = tree['input_parameters']['target']['crystal_characteristic_length'][()]
    delay_ = (radius_init - radius_) / 10. / (speed_of_disordering / 1e6)
    dst_ = {
        'timing_monitor': {
            'delay_time': {
                'value': delay_,
                'unit': 'femtosecond',
                'type': 'float'
            }
        }
    }
    return dst_

In [5]:
# conditions to generate raw Bragg spots
srcdir = "../output_data/01_simulation/"
srclist = sorted(glob.glob(os.path.join(srcdir, "*.h5")))

# output
outputdir = "../output_data/04_simulate_delay_time"
fname_fmt = 'delay_time_R{0:.2f}A.h5'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [6]:
speed_of_disordering = 70e3

radii = []
for fpath in srclist:
    with h5py.File(fpath, "r") as tree:
        radii.append(tree['input_parameters']['target']['crystal_characteristic_length'][()])

radii = np.array(radii)
radius_init = radii.max()

# main

In [7]:
for ii, srcpath in enumerate(srclist):
    radius_ = radii[ii]

    # calculate
    with h5py.File(srclist[ii], "r") as tree:
        dst = calculate_timing_monitor_info(tree, speed_of_disordering, radius_init)
    
        # output
        dstpath = os.path.join(outputdir, fname_fmt.format(radius_))
        output(dstpath, dst)

## check the output

In [ ]:
outputlist = sorted(glob.glob(os.path.join(outputdir, "*.h5")))

print_h5_tree_from_path(outputlist[0])

In [ ]:
index = 2

for fpath in outputlist:
    with h5py.File(fpath, "r") as tree:
        print(tree['timing_monitor']['delay_time'][()])